In [1]:
%pip install ../../../

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=67557 sha256=c9250a5325e05af6bba294e0424d2c550f5636095c7ed60f649ffd227f31c395
  Stored in directory: /tmp/pip-ephem-wheel-cache-y6h10d74/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="train",
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

In [4]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T


valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()

In [5]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [6]:
from spuco.models import model_factory 
model = model_factory("mlp", trainset[0][0].shape, trainset.num_spurious).to(device)
from torch.optim import SGD
from spuco.invariant_train import ERM

erm = ERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
erm.train()

Epoch 0: 100%|██████████| 751/751 [00:03<00:00, 245.65batch/s, accuracy=100.0%, loss=0.00422] 


In [7]:
from collections import Counter 

Counter(valset.spurious).most_common()

[(0, 2402), (1, 2401), (2, 2398), (3, 2398), (4, 2397)]

In [8]:
from spuco.group_inference import SSA
from spuco.utils import SpuriousTargetDataset

ssa = SSA(
    spurious_unlabeled_dataset=trainset,
    spurious_labeled_dataset=SpuriousTargetDataset(valset, valset.spurious),
    model=model_factory("lenet", trainset[0][0].shape, trainset.num_spurious).to(device),
    labeled_valset_size=0.1,
    num_iters=1000,
    tau_g_min=0.3,
    device=device,
    verbose=True
)

In [9]:
group_partition = ssa.infer_groups()
for key in sorted(group_partition.keys()):
    print(key, len(group_partition[key]))

Training SSA Model 0:   0%|          | 1/1000 [00:00<07:41,  2.16it/s, loss=nan, val_acc=19.5]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:7') tensor([2, 0, 1, 2, 1, 1, 3, 4, 2, 4, 4, 0, 4, 4, 0, 3, 2, 4, 0, 3, 0, 3, 1, 0,
        0, 2, 4, 2, 2, 4, 0, 3, 2, 2, 2, 0, 3, 1, 3, 4, 3, 2, 4, 4, 3, 3, 3, 2,
        0, 1, 0, 4, 2, 0, 1, 3, 0, 1, 3, 2, 1, 3, 0, 3])
g_min_indices tensor([], device='cuda:7', dtype=torch.int64)
group_indices tensor([], device='cuda:7', dtype=torch.int64)
group_indices tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63], device='cuda:7')
group_indices tensor([], device='cuda:7', dtype=torch.int64)
group_indices tensor([], device='cuda:7', dtyp

Training SSA Model 0:   1%|          | 11/1000 [00:00<00:46, 21.23it/s, loss=nan, val_acc=19.5]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:7') tensor([4, 4, 3, 1, 2, 4, 2, 1, 2, 3, 3, 1, 1, 4, 1, 1, 3, 1, 3, 0, 4, 4, 2, 0,
        1, 3, 2, 0, 2, 1, 3, 0, 3, 1, 0, 2, 0, 4, 1, 0, 0, 2, 0, 2, 3, 3, 2, 4,
        1, 4, 0, 0, 2, 3, 0, 1, 0, 4, 1, 0, 4, 4, 3, 1])
g_min_indices tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63], device='cuda:7')
group_indices tensor([], device='cuda:7', dtype=torch.int64)
group_indices tensor([], device='cuda:7', dtype=torch.int64)
group_indices tensor([], device='cuda:7', dtype=torch.int64)
group_indices tensor([], device='cuda:7', dtyp

Training SSA Model 0:   2%|▏         | 21/1000 [00:00<00:30, 32.11it/s, loss=nan, val_acc=36.9]

...

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:7') tensor([1, 1, 0, 1, 0, 4, 0, 2, 1, 2, 4, 4, 3, 4, 4, 3, 1, 2, 1, 3, 0, 3, 2, 2,
        3, 3, 1, 0, 0, 2, 0, 3, 4, 4, 1, 3, 3, 2, 4, 3, 3, 0, 1, 1, 2, 1, 3, 0,
        1, 3, 1, 2, 1, 1, 2, 3, 0, 4, 1, 1, 2, 2, 2, 1])
g_min_indices tensor([], device='cuda:7', dtype=torch.int64)
group_indices tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63], device='cuda:7')
group_indices tensor([], device='cuda:7', dtype=torch.int64)
group_indices tensor([], device='cuda:7', dtype=torch.int64)
group_indices tensor([], device='cuda:7', dtyp

Training SSA Model 2:  99%|█████████▉| 994/1000 [00:19<00:00, 52.29it/s, loss=3.3, val_acc=21.4] 

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:7') tensor([3, 2, 1, 0, 2, 2, 3, 0, 3, 3, 2, 2, 1, 3, 3, 0, 4, 2, 2, 1, 1, 0, 4, 0,
        4, 4, 1, 3, 2, 3, 3, 1, 3, 2, 4, 1, 4, 2, 0, 3, 0, 3, 3, 1, 1, 3, 0, 0,
        4, 0, 2, 1, 4, 0, 3, 4, 4, 0, 2, 0, 4, 2, 1, 1])
g_min_indices tensor([], device='cuda:7', dtype=torch.int64)
group_indices tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63], device='cuda:7')
group_indices tensor([], device='cuda:7', dtype=torch.int64)
group_indices tensor([], device='cuda:7', dtype=torch.int64)
group_indices tensor([], device='cuda:7', dtyp

Training SSA Model 2: 100%|██████████| 1000/1000 [00:19<00:00, 50.06it/s, loss=3.47, val_acc=21.4]


tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:7') tensor([1, 1, 2, 3, 3, 1, 4, 2, 0, 4, 0, 3, 1, 2, 2, 2, 3, 3, 0, 4, 0, 4, 4, 0,
        2, 1, 2, 0, 0, 1, 2, 4, 1, 3, 2, 0, 4, 4, 1, 4, 4, 1, 1, 4, 3, 0, 0, 1,
        2, 1, 2, 0, 3, 1, 0, 2, 4, 3, 4, 4, 1, 1, 1, 3])
g_min_indices tensor([], device='cuda:7', dtype=torch.int64)
group_indices tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63], device='cuda:7')
group_indices tensor([], device='cuda:7', dtype=torch.int64)
group_indices tensor([], device='cuda:7', dtype=torch.int64)
group_indices tensor([], device='cuda:7', dtyp

In [10]:
from spuco.evaluate import Evaluator 

evaluator = Evaluator(
    testset=trainset,
    group_partition=group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 100.0
Group (0, 1) Test Accuracy: 19.23076923076923
Group (0, 2) Test Accuracy: 0.0
Group (0, 3) Test Accuracy: 0.0
Group (0, 4) Test Accuracy: 0.0
Group (1, 0) Test Accuracy: 0.0
Group (1, 1) Test Accuracy: 100.0
Group (1, 2) Test Accuracy: 0.0
Group (1, 3) Test Accuracy: 0.0
Group (1, 4) Test Accuracy: 0.0
Group (2, 0) Test Accuracy: 0.0
Group (2, 1) Test Accuracy: 0.0
Group (2, 2) Test Accuracy: 100.0
Group (2, 3) Test Accuracy: 0.0
Group (2, 4) Test Accuracy: 0.0
Group (3, 0) Test Accuracy: 0.0
Group (3, 1) Test Accuracy: 0.0
Group (3, 2) Test Accuracy: 3.225806451612903
Group (3, 3) Test Accuracy: 100.0
Group (3, 4) Test Accuracy: 0.0
Group (4, 0) Test Accuracy: 0.0
Group (4, 1) Test Accuracy: 3.5714285714285716
Group (4, 2) Test Accuracy: 0.0
Group (4, 3) Test Accuracy: 0.0
Group (4, 4) Test Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 19.23076923076923,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 100.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 3.225806451612903,
 (3, 3): 100.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 3.5714285714285716,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 100.0}

In [11]:
from torch.optim import SGD
from spuco.invariant_train import GroupBalanceERM, ClassBalanceERM
from spuco.models import model_factory 

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
group_balance_erm = GroupBalanceERM(
    model=model,
    num_epochs=5,
    trainset=trainset,
    group_partition=group_partition,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
group_balance_erm.train()

Epoch 4: 100%|██████████| 751/751 [00:01<00:00, 397.69batch/s, accuracy=100.0%, loss=0.142]   


In [12]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 96.45390070921985
Group (0, 1) Test Accuracy: 85.57919621749409
Group (0, 2) Test Accuracy: 97.39952718676123
Group (0, 3) Test Accuracy: 87.70685579196217
Group (0, 4) Test Accuracy: 76.8321513002364
Group (1, 0) Test Accuracy: 92.42053789731051
Group (1, 1) Test Accuracy: 89.73105134474328
Group (1, 2) Test Accuracy: 89.2156862745098
Group (1, 3) Test Accuracy: 87.25490196078431
Group (1, 4) Test Accuracy: 82.3529411764706
Group (2, 0) Test Accuracy: 78.93333333333334
Group (2, 1) Test Accuracy: 58.666666666666664
Group (2, 2) Test Accuracy: 71.2
Group (2, 3) Test Accuracy: 63.2
Group (2, 4) Test Accuracy: 54.27807486631016
Group (3, 0) Test Accuracy: 93.46733668341709
Group (3, 1) Test Accuracy: 92.44332493702771
Group (3, 2) Test Accuracy: 92.44332493702771
Group (3, 3) Test Accuracy: 95.46599496221663
Group (3, 4) Test Accuracy: 96.4735516372796
Group (4, 0) Test Accuracy: 76.32241813602015
Group (4, 1) Test Accuracy: 60.70528967254408
Group (4, 2) Test

{(0, 0): 96.45390070921985,
 (0, 1): 85.57919621749409,
 (0, 2): 97.39952718676123,
 (0, 3): 87.70685579196217,
 (0, 4): 76.8321513002364,
 (1, 0): 92.42053789731051,
 (1, 1): 89.73105134474328,
 (1, 2): 89.2156862745098,
 (1, 3): 87.25490196078431,
 (1, 4): 82.3529411764706,
 (2, 0): 78.93333333333334,
 (2, 1): 58.666666666666664,
 (2, 2): 71.2,
 (2, 3): 63.2,
 (2, 4): 54.27807486631016,
 (3, 0): 93.46733668341709,
 (3, 1): 92.44332493702771,
 (3, 2): 92.44332493702771,
 (3, 3): 95.46599496221663,
 (3, 4): 96.4735516372796,
 (4, 0): 76.32241813602015,
 (4, 1): 60.70528967254408,
 (4, 2): 73.04785894206549,
 (4, 3): 69.1919191919192,
 (4, 4): 62.62626262626262}

In [13]:
evaluator.worst_group_accuracy

((2, 4), 54.27807486631016)

In [14]:
evaluator.average_accuracy

83.47851663560688

In [15]:
evaluator.evaluate_spurious_task()

21.08